**5주차 실습 - 카운트 기반 문서 표현을 직접 구현하기**

- 출처: 딥 러닝을 이용한 자연어 처리 입문(https://wikidocs.net/book/2155)

# **1. Bag of Words(BoW)**
- 각 단어에 고유한 정수 인덱스를 부여하여 벡터를 만들고 각 단어의 등장 횟수를 기록

In [2]:
from nltk.tokenize import TreebankWordTokenizer

def build_bag_of_words(doc):
    # 온점 제거 및 토큰화
    doc = doc.replace('.', '')
    tokenized_document = TreebankWordTokenizer().tokenize(doc)

    word_to_index = {}  # 단어와 해당 인덱스를 매핑할 딕셔너리
    bow = []  # BoW를 저장할 리스트

    for word in tokenized_document:
        if word not in word_to_index.keys():
            # 단어가 처음 등장한 경우, 인덱스를 부여하고 BoW에 1을 추가
            word_to_index[word] = len(word_to_index)
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 이미 등장한 단어의 경우, 해당 인덱스 위치의 BoW 값에 1을 추가
            index = word_to_index.get(word)
            bow[index] = bow[index] + 1

    return word_to_index, bow


In [4]:
doc1 = "Imagination is more important than knowledge is more"

vocab, bow = build_bag_of_words(doc1)

print('vocabulary :', vocab)
print('bag of words vector :', bow)


vocabulary : {'Imagination': 0, 'is': 1, 'more': 2, 'important': 3, 'than': 4, 'knowledge': 5}
bag of words vector : [1, 2, 2, 1, 1, 1]


In [5]:
doc2 = "Knowledge is limited"

vocab, bow = build_bag_of_words(doc2)

print('vocabulary :', vocab)
print('bag of words vector :', bow)


vocabulary : {'Knowledge': 0, 'is': 1, 'limited': 2}
bag of words vector : [1, 1, 1]


In [6]:
doc3 = "Imagination encircles the world"

vocab, bow = build_bag_of_words(doc3)

print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'Imagination': 0, 'encircles': 1, 'the': 2, 'world': 3}
bag of words vector : [1, 1, 1, 1]


In [7]:
doc4 = doc1 + " " + doc2 + " " + doc3

vocab, bow = build_bag_of_words(doc4)

print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'Imagination': 0, 'is': 1, 'more': 2, 'important': 3, 'than': 4, 'knowledge': 5, 'Knowledge': 6, 'limited': 7, 'encircles': 8, 'the': 9, 'world': 10}
bag of words vector : [2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1]


# **2. TF-IDF**

In [9]:
import pandas as pd
from math import log

doc1 = '먹고 싶은 사과'
doc2 = '먹고 싶은 바나나'
doc3 = '길고 노란 바나나 바나나'
doc4 = '저는 과일이 좋아요'

docs = [doc1, doc2, doc3, doc4]


# 총 문서의 수
D = len(docs)

# 문서 d 내에서 단어 t의 빈도(TF)를 계산
def tf(t, d):
    return d.count(t)

# 단어 t의 역문서 빈도(IDF)를 계산
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc  # doc에 t가 있는지 확인
    return log(D / (df + 1))
    # IDF의 다른 계산 방법
    # return log((D + 1) / (df + 1)) + 1

# TF-IDF 점수 계산
def tfidf(t, d):
    return tf(t, d) * idf(t)


In [11]:
# 문서 내에 등장하는 단어의 집합
vocas = list(set(w for doc in docs for w in doc.split()))
vocas.sort()

vocas

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [13]:
result = []  # 결과를 저장할 리스트

for doc in docs:
    result.append([])
    for voca in vocas:
        result[-1].append(tf(voca, doc))  # TF 값을 결과에 추가

tf_score = pd.DataFrame(result, columns=vocas)


In [14]:
tf_score

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [15]:
result = []  # 결과를 저장할 리스트

for voca in vocas:
    result.append(idf(voca))  # IDF 값을 결과에 추가

idf_score = pd.DataFrame(result, index=vocas, columns=["IDF"])


In [16]:
idf_score

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [17]:
result = []  # 결과를 저장할 리스트

for doc in docs:
    result.append([])
    for voca in vocas:
        result[-1].append(tfidf(voca, doc))  # TF-IDF 값을 결과에 추가

tfidf_score = pd.DataFrame(result, columns=vocas)


In [18]:
tfidf_score

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


# **3. 코사인 유사도(Cosine Similarity)**

In [19]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서1과 문서2의 유사도 : 0.6666666666666667
문서1과 문서3의 유사도 : 0.6666666666666667
문서2와 문서3의 유사도 : 1.0000000000000002


In [21]:
# 과일 텍스트 예시의 TF-IDF를 이용한 코사인 유사도 계산
doc1 = tfidf_score.iloc[0]
doc2 = tfidf_score.iloc[1]
doc3 = tfidf_score.iloc[2]
doc4 = tfidf_score.iloc[2]

print('문서1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서1과 문서4의 유사도 :',cos_sim(doc1, doc4))

문서1과 문서2의 유사도 : 0.4133084876261022
문서1과 문서3의 유사도 : 0.0
문서1과 문서4의 유사도 : 0.0
